# 连接数据库

In [1]:
import pymysql

# 与数据库建立连接
connect_obj = pymysql.connect(host='localhost', user='root', password='1231', database='mydata_firsttry', port=3306)
cur = connect_obj.cursor()  # 获取游标

# 创建数据库表

* CREATE TABLE 语句
  
  创建数据库表，格式：
  
  CREATE TABLE 表名(
  
  列名1 数据类型
  
  列名2 数据类型
  
  ...
  
  );
  
* 常见的一些sql约束

  NOT NULL - 指示某列不能存储 NULL 值。
  
  UNIQUE - 保证某列的每行必须有唯一的值。

  PRIMARY KEY - NOT NULL 和 UNIQUE 的结合。确保某列（或两个列多个列的结合）有唯一标识，有助于更容易更快速地找到表中的一个特定的记录。

  FOREIGN KEY - 保证一个表中的数据匹配另一个表中的值的参照完整性。
 
  CHECK - 保证列中的值符合指定的条件。

  DEFAULT - 规定没有给列赋值时的默认值。
  
* 注意**使用pymysql创建数据库表，不用像修改表中数据那样最后要commit**

In [2]:
sql = """
    CREATE TABLE USER (
    id INT auto_increment PRIMARY KEY ,
    name CHAR(10) NOT NULL UNIQUE,
    age TINYINT NOT NULL
    )
"""
try:
    cur.execute(sql)
except Exception as e:
    print(e)
    # 发生错误则回滚
    connect_obj.rollback()

#  INSERT INTO 语句

**向表中增加数据**

* 格式：

  INSERT INTO 表名(列名1， 列名2， 列名3...)
  
  VALUES(%s, %s, %s...);
  
* 注意：
  
  **VALUES统一使用%s**
  
  **对数据库表进行修改要commit提交修改**
  
---
 
### 下面是对多个数据进行插入executemany方法

In [3]:
# 增:insert into,id是主键自增
sql = """
    INSERT INTO USER(name, age)
    VALUES(%s, %s)
"""
data = [
    ("Mr.Gu", 20),
    ("Mr.Li", 20),
    ("Mr.Lin", 20),
    ("Mr.Huang", 17)
]
try:
    # 执行sql语句
    cur.executemany(sql, data)
    # 提交执行
    connect_obj.commit()
except Exception as e:
    print(e)
    # 发生错误则回滚
    connect_obj.rollback()

###  对单个数据进行插入

* 方法execute
  
  可以将单个数据写进变量sql，也可以像上面的多值插入一样方法中传入两个变量，一个sql语句，一个数据

In [4]:
sql = "\
    INSERT INTO USER(name, age) \
    VALUES('%s', %s)" % ("Mr.unknow", 19)

try: 
    cur.execute(sql)
    connect_obj.commit()
except Exception as e:
    print(e)
    connect_obj.rollback()

# SELECT 语句

**从表中选取数据**

* 格式：

  1. "SELECT 列名1, 列名2... FROM 表名;"
  
  这种按照列取数据
  
  2. "SELECT * FROM 表名;"
  
  全部选择

### WHERE 子句

**WHERE 子句用于提取那些满足指定条件的记录**

* 格式：
  
  "SELECT * FROM 表名 WHERE 列名 条件;"
  
前面选取方法和上面一样还是有两种，然后在后面添加上需要的条件

下面的例子中条件就是： age这个数据要等于20

### SELECT DISTINCT 语句

**SELECT DISTINCT 语句用于返回唯一不同的值。**

* 格式：
  
  SELECT DISTINCT 列名 FROM 表名;
  
DISTINCT加在选择的列前面

如果列名这里改成*试一下好像不行

下面给出一个例子

In [5]:
# 查询:select
sql = "SELECT * FROM USER \
       WHERE age = %d" % (20)
try:
   # 执行SQL语句
   cur.execute(sql)
   # 获取所有记录列表
   results = cur.fetchall()
   print(results)
   for row in results:
      id = row[0]
      name = row[1]
      age = row[2]
       # 打印结果
      print ("id:%s,name:%s,age=%d" % \
              (id, name, age))
except Exception as e:
    print(e)
    print ("Error: unable to fetch data")

((1, 'Mr.Gu', 20), (2, 'Mr.Li', 20), (3, 'Mr.Lin', 20))
id:1,name:Mr.Gu,age=20
id:2,name:Mr.Li,age=20
id:3,name:Mr.Lin,age=20


In [6]:
sql = "SELECT DISTINCT AGE FROM USER "

try:
    cur.execute(sql)
    results = cur.fetchall()
    print(results)
except Exception as e:
    print(e)


((20,), (17,), (19,))


# ORDER BY 关键字

**ORDER BY 关键字用于对结果集进行排序。**

* 格式：

  SELECT *

  FROM 表名

  ORDER BY 列名1，列名2... ASC|DESC;
  
* 注意：

  1.**ASC为升序，DESC为降序**

  2.**多字段排序时会按照顺序来，例如我by后面跟是age grade，那么它会先按照age排序，age中有相同的，那么些相同的部分按照grade排序**
  
下面是一个排序的例子

In [7]:
sql = "SELECT * FROM USER\
       ORDER BY AGE ASC"

try:
    cur.execute(sql)
    results = cur.fetchall()
    print(results)
except Exception as e:
    print(e)

((4, 'Mr.Huang', 17), (5, 'Mr.unknow', 19), (1, 'Mr.Gu', 20), (2, 'Mr.Li', 20), (3, 'Mr.Lin', 20))


# AND & OR 运算符

运算符，用法和平时差不多

可以用于where后面的条件约束中

栗子：

In [8]:
sql = "SELECT * FROM USER \
       WHERE age = %d or age = %d" % (17, 19)  # 取出年龄为19和17的样本

try:
    cur.execute(sql)
    results = cur.fetchall()
    print(results)
except Exception as e:
    print(e)

((4, 'Mr.Huang', 17), (5, 'Mr.unknow', 19))


# UPDATE 语句

**UPDATE 语句用于更新表中已存在的记录。**

* 格式：

  UPDATE 表名 
  
  SET 列名 = 数据... WHERE 列名  约束条件;
  
* 注意：**如果不加where约束，那么会更新所有数据**

In [9]:
# 改：UPDATE
sql = "UPDATE USER SET AGE = AGE + 1 WHERE name = '%s'" % ('Mr.Gu')
try:
   # 执行SQL语句
   cur.execute(sql)
   # 提交到数据库执行
   connect_obj.commit()
except Exception as e:
   print(e)
   # 发生错误时回滚
   connect_obj.rollback()

# DELETE 语句

**DELETE 语句用于删除表中的行。**
<font color=orange>**注意这里删除的是行**</font>

* 格式：

  DELETE 表名 
  
  SET 列名 = 数据... WHERE 列名  约束条件;
  
和上面的更新差不多

In [10]:
# 删：DELETE
sql = "DELETE FROM USER WHERE AGE = %s" % (19)
try:
   # 执行SQL语句
   cur.execute(sql)
   # 提交修改
   connect_obj.commit()
except Exception as e:
    print(e)
   # 发生错误时回滚
    connect_obj.rollback()

In [11]:
# 关闭数据库连接
connect_obj.close()